# Download Strava Activity Summaries for Specified Athlete

This short script will download all Strava activity (aka ride) summaries for a given athlete. The main use for this script is to download a list of activity IDs for an athlete. We can use those activity IDs to download details for a given activity.

In [ ]:
#imports stravalib library
from stravalib import Client

#other libraries
import csv

In [ ]:
#note: these tokens are assigned by Strava on a per-developer, per-athlete basis.
#the process is for the developer to create an 'application' and then for the athlete
#to authenticate to that application. once the athlete authenticates to the application
#Strava will return to the developer a token that the developer can use to access
#the athlete's data

client = Client(access_token=[REDACTED])        #no need to publish this to the whole world

athleteid = client.get_athlete().id

In [ ]:
ofile = open('./Data/' + str(athleteid) + '_ActivitiesList.txt', 'w')
writer = csv.writer(ofile, delimiter='|', quotechar='^', quoting=csv.QUOTE_ALL)

#write header row for text file
header = ['AthleteID', 'RideID', 'RideDate', 'RideName', 'Distance', 'Total_Elevation_Gain', 'Lat', 'Lon']
writer.writerow(header);

In [ ]:
def get_activities():
    return list(client.get_activities())

In [ ]:
def build_tuple(activityid):
  strava_activity = client.get_activity(activityid)
  if strava_activity.type == "Ride" and strava_activity.trainer == False and strava_activity.commute == False \
    and strava_activity.start_latlng is not None:
    lat = round(strava_activity.start_latlng[0],3)
    lng = round(strava_activity.start_latlng[1],3)
    latlng = round((lat+90)*180+lng,3)  #combines lat/lng into a single number. useful for grouping rides by lat/lng
    ride_date = strava_activity.start_date_local.date()
    miles_ridden = format(float(strava_activity.distance * 0.00062137), '.1f')             #convert meters to miles
    feet_climbed = format(float(strava_activity.total_elevation_gain * 3.2808399), '.0f')  #convert meters to feet
    segment_tuple = athleteid, strava_activity.id, ride_date, \
            ascii(strava_activity.name), miles_ridden, feet_climbed, lat, lng, latlng  
    return segment_tuple

In [ ]:
%%capture --no-stdout
#stravalib generates a lot of warning messages that can be safely discarded
num_rides = 0

strava_activities = get_activities()

for j in range(len(strava_activities)):
    activityid = strava_activities[j].id
    segment_tup = build_tuple(activityid)
    if type(segment_tup) != NoneType:
        writer.writerow(segment_tup) 
    num_rides +=1
    if num_rides % 10 == 0:
        print('Downloaded {0} rides out of {1} so far...'.format(num_rides, len(strava_activities)))
        
print('All done! Finished downloading {0} rides'.format(num_rides))

In [ ]:
ofile.close()